In [1]:
import npu
from npu.vision.datasets import CIFAR10
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# TODO: Replace YOUR_TOKEN with your API token
npu.api('LsqV0Z2PP-_A_tceb_Mtkx16i1r1knOrxW3_Q0b-HD8', project='get_started')

[Neuro Ai] 09:39:22 - [INFO]: Token successfully authenticated
[Neuro Ai] 09:39:22 - [INFO]: Using project: get_started


In [3]:
class Net(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [4]:
model = npu.compile(net)

[Neuro Ai] 09:39:25 - [INFO]: Model file size is 247.63 KB. Uploading model now...


[Neuro Ai] Uploading: 100%|██████████████████████████████████████████████████| 248k/248k [00:00<00:00, 2.65MB/s]


[Neuro Ai] 09:39:25 - [INFO]: Started compiling. View status at https://dashboard.getneuro.ai/task?id=614856cb256db7477622b0c0


[Neuro Ai] COMPILING: 100.0%|██████████████████████████████████████████████| remaining: 00:00 || elapsed: 00:00 

[Neuro Ai] 09:39:25 - [INFO]: Model compiled successfully.


In [5]:
trained_model = npu.train(model,
                          train_data=CIFAR10.train,
                          val_data=CIFAR10.val,
                          loss=npu.loss.SparseCrossEntropyLoss,
                          optim=npu.optim.SGD(lr=0.01),
                          batch_size=256,
                          epochs=10)

[Neuro Ai] 09:40:32 - [INFO]: Started training. View status at https://dashboard.getneuro.ai/task?id=6148570d7e79573bf7306034


[Neuro Ai] TRAINING: 100.0%|███████████████████████████████████████████████| remaining: 00:00 || elapsed: 00:29 

[Neuro Ai] 09:41:01 - [INFO]: Model finished training


In [6]:
import numpy as np
from tensorflow.keras.datasets import cifar10
_, (x_test, y_test) = cifar10.load_data()

# Since the global models are in pytorch, we need to make sure the shape of our prediction data is [N, C, H, W]
x_test = x_test.transpose((0, 3, 2, 1)).astype(float) # [1000, 3, 32, 32]

# Get 100 random sample
n = np.random.randint(0, 500)
x = x_test[n:n+100]

2021-09-20 10:41:59.163473: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2021-09-20 10:41:59.163523: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
# Use trained_model or "MODEL_ID" found in your dashboard
predictions = npu.predict(model=trained_model, data=x).get_result()[0]
# predictions = npu.predict(model='6148532aa7cde87018712eb1', data=x).get_result()[0]

[Neuro Ai] 09:42:03 - [INFO]: Data not found on servers. Total size of data is 2.34 MB. Uploading now...


[Neuro Ai] Uploading: 100%|████████████████████████████████████████████████| 2.34M/2.34M [00:01<00:00, 1.79MB/s]


[Neuro Ai] 09:42:05 - [INFO]: Started prediction. View status at https://dashboard.getneuro.ai/task?id=6148576b256db7477622b0fb


[Neuro Ai] PREDICTION: 100.0%|█████████████████████████████████████████████| remaining: 00:00 || elapsed: 00:00 
[Neuro Ai] PREDICTION: 100.0%|█████████████████████████████████████████████| remaining: 00:00 || elapsed: 00:00 


In [8]:
accuracy = 0
for i, result in enumerate(predictions):
    predicted = np.argmax(result)
    if y_test[n+i] == predicted:
        accuracy+=1
print(accuracy, "correct out of", len(x))

43 correct out of 100
